In [10]:
import json
import os
import sys
import tensorflow as tf
import numpy as np

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
def createModel(vocab_size, batchSize):
    #Document
    document_input = tf.keras.layers.Input(batch_shape = [batchSize, None])
    document = tf.keras.layers.Embedding(vocab_size, 256, batch_input_shape = [batchSize, None])(document_input)
    document = tf.keras.layers.GRU(1024, return_sequences = True, stateful = True, recurrent_initializer='glorot_uniform')(document)
    document = tf.keras.layers.Dense(256, activation = 'relu')(document)
    document_model = tf.keras.models.Model(inputs = document_input, outputs = document)
    
    #Question
    question_input = tf.keras.layers.Input(batch_shape = [batchSize, None])
    question = tf.keras.layers.Embedding(vocab_size, 256)(question_input)
    question = tf.keras.layers.GRU(1024, return_sequences = True, stateful = True, recurrent_initializer='glorot_uniform')(question)
    question = tf.keras.layers.Dense(126, activation = 'relu')(question)
    question_model = tf.keras.models.Model(inputs = question_input, outputs = question)
    
    #concat
    model = tf.keras.layers.concatenate([document_model.output, question_model.output])
    model = tf.keras.layers.Dense(vocab_size)(model)
    
    finalModel = tf.keras.models.Model(inputs = [document_input, question_input], outputs = model)
    
    return finalModel

In [18]:
class trainGenSeq_short(tf.keras.utils.Sequence):
    def __init__(self, batchSize):
        self.batchSize = batchSize
        
        self.trainFiles = os.listdir('D:/Python/Datasets/v1.0/train/')
        #char2idx, 
        self.vocab_Dict, self.vocab_Array = self.loadVocab()
        self.trainingSamples = 307372
    
    def __len__(self):
        return int(self.trainingSamples // self.batchSize)
    
    def getLen(self):
        return int(self.trainingSamples // self.batchSize)
    
    def loadVocab(self):
        myList = []
        for line in open('dict_list.txt'):
            myList.append(line)
        v = set(myList)
        
        a = {word:index for index, word in enumerate(v)}
        b = np.array(v)
        return a,b
    
    def __getitem__(self, trash):
        documentStack = []
        titleStack = []
        questionStack = []
        answerStack = []
        
        for file in self.trainFile:
            for line in open('D:/Python/Datasets/v1.0/train/' + file):
                file = json.loads(line)
                
                question = file.get('question_text')
                title = file.get('document_title')
                
                #annotations
                s_Start = file.get('annotations')[0].get('short_answers')[0].get('start_token')
                s_End = file.get('annotations')[0].get('short_answers')[0].get('end_token')
                
                #document
                document = []

                for indexs in file.get('document_tokens'):
                    if indexs.get('html_token') == False:
                        document.append(indexs.get('token'))
                
                #answer
                answer = []
                
                for index in range(s_Start,s_End):
                    if file.get('document_tokens')[index].get('html') == False:
                        answer.append(file.get('document_tokens')[index].get('token'))
                
                #Convert to Array
                question = np.array([self.vocab_Dict[i] for i in question])
                title = np.array([self.vocab_Dict[i] for i in title])
                document = np.array([self.vocab_Dict[i] for i in ' '.join(document)])
                answer = np.array([self.vocab_Dict[i] for i in ' '.join(answer)])
                
            documentStack.append()
            titleStack.append(title)
            questionStack.append(question)
            answerStack.append()
            
            if len(documentStack) == self.batchSize:
                yield [np.array(documentStack), np.array(questionStack)], np.array(answerStack)
                
                

In [19]:
batchSize = 64
vocabSize = 20543

model = createModel(vocabSize, batchSize)
model.summary()

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (64, None, 256)      5259008     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (64, None, 256)      5259008     input_10[0][0]                   
___________________________________________________________________________________________

In [21]:
trainSeq = trainGenSeq_short(batchSize)

model.fit(   trainSeq,
             epochs = 2,
             steps_per_epoch = trainSeq.getLen(),
             verbose = 1)

AttributeError: 'generator' object has no attribute 'dtype'